### Merge data from different lanes and remove doublet (patient 5,6,7,8)

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import sys
import bbknn
from statsmodels import robust
import matplotlib.pyplot as plt
import os.path

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
#sc.settings.set_figure_params(dpi=80, color_map='viridis')
sc.logging.print_versions()

The `sinfo` package has changed name and is now called `session_info` to become more discoverable and self-explanatory. The `sinfo` PyPI package will be kept around to avoid breaking old installs and you can downgrade to 0.3.2 if you want to use it without seeing this message. For the latest features and bug fixes, please install `session_info` instead. The usage and defaults also changed slightly, so please review the latest README at https://gitlab.com/joelostblom/session_info.
-----
anndata     0.7.6
scanpy      1.8.1
sinfo       0.3.4
-----
PIL                 8.1.2
annoy               NA
backcall            0.2.0
bbknn               NA
cffi                1.15.0
cloudpickle         2.1.0
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dask                2022.02.0
dateutil            2.8.1
decorator           4.4.2
fsspec              2021.10.0
google              NA
h5py                2.10.0
igraph              0.8.0
ipykernel           5.5.0
ipython_

In [11]:
pip list

Package                       Version
----------------------------- -------------------
absl-py                       0.14.1
adjustText                    0.8b2
aiohttp                       3.7.4.post0
airr                          1.3.1
anndata                       0.7.6
anndata2ri                    1.0.6
annoy                         1.17.0
argon2-cffi                   21.1.0
async-timeout                 3.0.1
attrs                         21.2.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.1
backports.zoneinfo            0.2.1
bbknn                         1.4.1
bleach                        4.1.0
bokeh                         2.4.3
cached-property               1.5.2
cachetools                    4.2.4
celltypist                    1.0.0
certifi                       2020.12.5
cffi                          1.15.0
chardet                       4.0.0
charset-normalizer            2.0.6
click                         8.0.1
cloudpickle                   2.1

In [47]:
path = "/lustre/scratch117/cellgen/team205/rl20/CTCL/Cellranger_out"
manifest = pd.read_csv(os.path.join(path, 'sample_manifest.csv'))
manifest


,sample_id,donor_id,Tissue,id
0,CTCL5_Derm_45N_G,CTCL5,Dermis,18
1,CTCL5_Derm_45P_8N_G,CTCL5,Dermis,19
2,CTCL5_Derm_45P_8Pr_G,CTCL5,Dermis,20
3,CTCL5_Epi_45N_G,CTCL5,Epidermis,21
4,CTCL5_Epi_45P_8N_G,CTCL5,Epidermis,22
5,CTCL5_Epi_45P_8Pr_G,CTCL5,Epidermis,23
6,CTCL6_Derm45N_G,CTCL6,Dermis,24
7,CTCL6_Derm45P_8N_G,CTCL6,Dermis,25
8,CTCL6_Derm45P_8Pr_G,CTCL6,Dermis,26
9,CTCL6_Epi45N_G,CTCL6,Epidermis,27


In [48]:
names = manifest['sample_id'].tolist()
filenames = [os.path.join(path, n, 'filtered_feature_bc_matrix') for n in names]


In [49]:
ID = manifest['id'].tolist()
ID = list(map(str, ID))

In [11]:
adatas = [sc.read_10x_mtx(filename, cache=True) for filename in filenames]

... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedu

In [62]:
adata = adatas[0].concatenate(adatas[1:], join='inner', batch_key='ID', batch_categories=ID, index_unique='-')


In [63]:
manifest

,sample_id,donor_id,Tissue,id
0,CTCL5_Derm_45N_G,CTCL5,Dermis,18
1,CTCL5_Derm_45P_8N_G,CTCL5,Dermis,19
2,CTCL5_Derm_45P_8Pr_G,CTCL5,Dermis,20
3,CTCL5_Epi_45N_G,CTCL5,Epidermis,21
4,CTCL5_Epi_45P_8N_G,CTCL5,Epidermis,22
5,CTCL5_Epi_45P_8Pr_G,CTCL5,Epidermis,23
6,CTCL6_Derm45N_G,CTCL6,Dermis,24
7,CTCL6_Derm45P_8N_G,CTCL6,Dermis,25
8,CTCL6_Derm45P_8Pr_G,CTCL6,Dermis,26
9,CTCL6_Epi45N_G,CTCL6,Epidermis,27


In [64]:
#%% Metadata
for metadata in manifest.columns[0:-1]: # sample_id and vdj not needed
    print(metadata)
    adata.obs[metadata] = adata.obs['ID']
    replacement = manifest[metadata].tolist()
    adata.obs[metadata].replace(to_replace=ID, value=replacement, inplace=True)
    

sample_id
donor_id
Tissue


In [66]:
del(adata.obs['ID'])

In [69]:
adata.obs

,sample_id,donor_id,Tissue
AAACCTGAGCTGATAA-1-18,CTCL5_Derm_45N_G,CTCL5,Dermis
AAACCTGAGTCATCCA-1-18,CTCL5_Derm_45N_G,CTCL5,Dermis
AAACCTGCAAGCCGTC-1-18,CTCL5_Derm_45N_G,CTCL5,Dermis
AAACCTGCACCGTTGG-1-18,CTCL5_Derm_45N_G,CTCL5,Dermis
AAACCTGGTCCCTACT-1-18,CTCL5_Derm_45N_G,CTCL5,Dermis
...,...,...,...
TTTGTCATCAACGCTA-1-40,CTCL8_Epi45P_POOL_G,CTCL8,Epidermis
TTTGTCATCACGCGGT-1-40,CTCL8_Epi45P_POOL_G,CTCL8,Epidermis
TTTGTCATCCAGATCA-1-40,CTCL8_Epi45P_POOL_G,CTCL8,Epidermis
TTTGTCATCGGTCTAA-1-40,CTCL8_Epi45P_POOL_G,CTCL8,Epidermis


In [70]:
save_file = '/lustre/scratch117/cellgen/team205/rl20/CTCL/object-new/CTCL5678_raw.h5ad'
adata.write(save_file)


... storing 'sample_id' as categorical
... storing 'donor_id' as categorical
... storing 'Tissue' as categorical
... storing 'feature_types' as categorical


In [6]:
input_file = '/lustre/scratch117/cellgen/team205/rl20/CTCL/object-new/CTCL5678_raw.h5ad'
adata = sc.read_h5ad(input_file)

In [7]:
adata.var

,gene_ids,feature_types
MIR1302-2HG,ENSG00000243485,Gene Expression
FAM138A,ENSG00000237613,Gene Expression
OR4F5,ENSG00000186092,Gene Expression
AL627309.1,ENSG00000238009,Gene Expression
AL627309.3,ENSG00000239945,Gene Expression
...,...,...
AC233755.2,ENSG00000277856,Gene Expression
AC233755.1,ENSG00000275063,Gene Expression
AC240274.1,ENSG00000271254,Gene Expression
AC213203.1,ENSG00000277475,Gene Expression


In [8]:
adata.var['gene'] = adata.var.index
adata.var.index = adata.var['gene_ids'].copy()
adata.var_names = adata.var['gene_ids'].copy()
adata.var.index.name = None
adata.var

,gene_ids,feature_types,gene
ENSG00000243485,ENSG00000243485,Gene Expression,MIR1302-2HG
ENSG00000237613,ENSG00000237613,Gene Expression,FAM138A
ENSG00000186092,ENSG00000186092,Gene Expression,OR4F5
ENSG00000238009,ENSG00000238009,Gene Expression,AL627309.1
ENSG00000239945,ENSG00000239945,Gene Expression,AL627309.3
...,...,...,...
ENSG00000277856,ENSG00000277856,Gene Expression,AC233755.2
ENSG00000275063,ENSG00000275063,Gene Expression,AC233755.1
ENSG00000271254,ENSG00000271254,Gene Expression,AC240274.1
ENSG00000277475,ENSG00000277475,Gene Expression,AC213203.1


In [9]:
adata.var.to_csv('/home/jovyan/farm/gene_ID_ENSG_table.csv')

In [4]:
adata.obs['donor_id'].value_counts()

CTCL7    82203
CTCL5    44882
CTCL6    40151
CTCL8    39126
Name: donor_id, dtype: int64

#### doublet removal

In [71]:
meta_10x_channels = 'sample_id'


In [72]:
RUNs, DSs, CELLs, THRs, MEDs, MADs, CUTs, no_thr = [], [], [], [], [], [], [], []

for run in adata.obs[meta_10x_channels].unique():
    print(run)
    ad = adata[adata.obs[meta_10x_channels] == run, :]
    x = ad.X
    scrub = scr.Scrublet(x)
    ds, prd = scrub.scrub_doublets()
    RUNs.append(run)
    DSs.append(ds)
    CELLs.append(ad.obs_names)
    # MAD calculation of threshold:
    MED = np.median(ds)
    MAD = robust.mad(ds)
    CUT = (MED + (4 * MAD))
    MEDs.append(MED)
    MADs.append(MAD)
    CUTs.append(CUT)

    try:  # not always can calculate automatic threshold
        THRs.append(scrub.threshold_)
        print('Threshold found by scrublet')
    except:
        THRs.append(0.4)
        no_thr.append(run)
        print('No threshold found, assigning 0.4 to', run)
        scrub.call_doublets(threshold=0.4) # so that it can make the plot
    fig = scrub.plot_histogram()
    fig[0].savefig(run + '.png')

    # Alternative histogram for MAD-based cutoff
    scrub.call_doublets(threshold=CUT)
    fig = scrub.plot_histogram()
    fig[0].savefig(run + '_mad_' + '.png')
    plt.close('all')
    print()
    print()

CTCL5_Derm_45N_G
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.35
Detected doublet rate = 2.7%
Estimated detectable doublet fraction = 28.8%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 9.3%
Elapsed time: 22.9 seconds
Threshold found by scrublet
Detected doublet rate = 5.4%
Estimated detectable doublet fraction = 43.5%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 12.4%


CTCL5_Derm_45P_8N_G
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.75
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.5%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 6.1%
Elapsed time: 12.8 seconds
Threshold found by scrublet
Detected doublet rate = 3.1%
Estimated detectable doublet fraction = 26.5%
Overall doublet rate:
	Expected   = 10.0%
	Estim

In [73]:
ns = np.array(list(map(len, DSs)))

In [74]:
tbl = pd.DataFrame({
    'run': np.repeat(RUNs, ns),
    'ds': np.concatenate(DSs),
    'thr': np.repeat(THRs, ns),
    'mad_MED': np.repeat(MEDs, ns),
    'mad_MAD': np.repeat(MADs, ns),
    'mad_thr': np.repeat(CUTs, ns),
    }, index=np.concatenate(CELLs))

tbl['auto_prd'] = tbl['ds'] > tbl['thr']
tbl['mad_prd'] = tbl['ds'] > tbl['mad_thr']

In [75]:
tbl.to_csv('/lustre/scratch117/cellgen/team205/rl20/CTCL/object-new/CTCL5678_doublets_score_mad.csv', header=True, index=True)


In [76]:
adata.obs['mad_prd']=tbl['mad_prd']
adata.obs['ds']=tbl['ds']
adata.obs['mad_MED']=tbl['mad_MED']
adata.obs['mad_MAD']=tbl['mad_MAD']
adata.obs['mad_thr']=tbl['mad_thr']
adata = adata[tbl['mad_prd'] != True]


In [77]:
mito_genes = adata.var_names.str.startswith('MT-')
adata.obs['percent_mito'] = np.sum(adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
adata.obs['n_counts'] = adata.X.sum(axis=1).A1


Trying to set attribute `.obs` of view, copying.


In [80]:
del(adata.obs['percent_mito'])
del(adata.obs['n_counts'])

In [81]:
adata.obs

,sample_id,donor_id,Tissue,mad_prd,ds,mad_MED,mad_MAD,mad_thr
AAACCTGAGCTGATAA-1-18,CTCL5_Derm_45N_G,CTCL5,Dermis,False,0.129032,0.059282,0.036473,0.205175
AAACCTGAGTCATCCA-1-18,CTCL5_Derm_45N_G,CTCL5,Dermis,False,0.107930,0.059282,0.036473,0.205175
AAACCTGCAAGCCGTC-1-18,CTCL5_Derm_45N_G,CTCL5,Dermis,False,0.093656,0.059282,0.036473,0.205175
AAACCTGCACCGTTGG-1-18,CTCL5_Derm_45N_G,CTCL5,Dermis,False,0.062500,0.059282,0.036473,0.205175
AAACCTGGTTGCCTCT-1-18,CTCL5_Derm_45N_G,CTCL5,Dermis,False,0.038610,0.059282,0.036473,0.205175
...,...,...,...,...,...,...,...,...
TTTGTCATCAACGCTA-1-40,CTCL8_Epi45P_POOL_G,CTCL8,Epidermis,False,0.094675,0.061414,0.033348,0.194805
TTTGTCATCACGCGGT-1-40,CTCL8_Epi45P_POOL_G,CTCL8,Epidermis,False,0.061414,0.061414,0.033348,0.194805
TTTGTCATCCAGATCA-1-40,CTCL8_Epi45P_POOL_G,CTCL8,Epidermis,False,0.048705,0.061414,0.033348,0.194805
TTTGTCATCGGTCTAA-1-40,CTCL8_Epi45P_POOL_G,CTCL8,Epidermis,False,0.032599,0.061414,0.033348,0.194805


In [82]:
save_file = '/lustre/scratch117/cellgen/team205/rl20/CTCL/object-new/CTCL5678_raw_rmdou.h5ad'
adata.write(save_file)
